In [ ]:
#Q-1

import requests
from bs4 import BeautifulSoup

def search_amazon(product):
    base_url = f"https://www.amazon.in/s?k={product}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        for product in products:
            product_name = product.find('span', {'class': 'a-size-medium'}).text.strip()
            product_price = product.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).text.strip()

            print("Product:", product_name)
            print("Price:", product_price)
            print("------")
    else:
        print("Failed to fetch the data.")

if __name__ == "__main__":
    user_input = input("Enter the product you want to search for on Amazon.in: ")
    search_amazon(user_input)


In [ ]:
# Q-2
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_product_details(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(product_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        brand_name = soup.find('a', {'id': 'bylineInfo'}).text.strip() if soup.find('a', {'id': 'bylineInfo'}) else "-"
        product_name = soup.find('span', {'id': 'productTitle'}).text.strip() if soup.find('span', {'id': 'productTitle'}) else "-"
        price = soup.find('span', {'id': 'priceblock_ourprice'}).text.strip() if soup.find('span', {'id': 'priceblock_ourprice'}) else "-"
        return_exchange = soup.find(text='Return Policy:').find_next('span').text.strip() if soup.find(text='Return Policy:') else "-"
        expected_delivery = soup.find(text='Arrives:').find_next('span').text.strip() if soup.find(text='Arrives:') else "-"
        availability = soup.find('div', {'id': 'availability'}).text.strip() if soup.find('div', {'id': 'availability'}) else "-"
        
        return {
            "Brand Name": brand_name,
            "Name of the Product": product_name,
            "Price": price,
            "Return/Exchange": return_exchange,
            "Expected Delivery": expected_delivery,
            "Availability": availability,
            "Product URL": product_url
        }
    else:
        return {}

def search_amazon(product):
    base_url = f"https://www.amazon.in/s?k={product}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    products_data = []

    for page in range(1, 4):  # Fetch first 3 pages or available pages
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            products = soup.find_all('div', {'data-component-type': 's-search-result'})
            
            for product in products:
                product_link = product.find('a', {'class': 'a-link-normal a-text-normal'})['href']
                product_url = f"https://www.amazon.in{product_link}"
                product_details = get_product_details(product_url)
                products_data.append(product_details)

    df = pd.DataFrame(products_data)
    df.to_csv(f"{product}_details.csv", index=False)

if __name__ == "__main__":
    user_input = input("Enter the product you want to search for on Amazon.in: ")
    search_amazon(user_input)


In [ ]:
#Q-3

pip install selenium

from selenium import webdriver
import time
import requests
import os

def fetch_images(search_query, num_images):
    driver = webdriver.Chrome(executable_path='path_to_chromedriver')  # Set the path to your chromedriver
    driver.get('https://www.google.com/imghp?hl=en')

    search_bar = driver.find_element_by_name('q')
    search_bar.send_keys(search_query)
    search_bar.submit()

    # Scroll to load more images (change the range to load more images if needed)
    for _ in range(10):
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight)")
        time.sleep(2)

    # Get image elements
    images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
    image_urls = set()

    for image in images:
        image.click()
        time.sleep(2)
        original_images = driver.find_elements_by_xpath('//img[@class="n3VNCb"]')
        for img in original_images:
            if img.get_attribute('src') and 'http' in img.get_attribute('src'):
                image_urls.add(img.get_attribute('src'))
            if len(image_urls) >= num_images:
                break
        if len(image_urls) >= num_images:
            break

    driver.quit()

    # Saving images to a folder
    folder_name = f"{search_query}_images"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for i, url in enumerate(image_urls):
        response = requests.get(url)
        file = open(f"{folder_name}/image_{i + 1}.jpg", "wb")
        file.write(response.content)
        file.close()

# Keywords to search for
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

# Fetch 10 images for each keyword
for keyword in keywords:
    fetch_images(keyword, 10)


In [ ]:
#Q-4
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_flipkart_smartphones(product):
    base_url = f"https://www.flipkart.com/search?q={product}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        products = soup.find_all('div', {'class': '_1AtVbE'})

        smartphone_data = []
        for product in products:
            try:
                product_url = "https://www.flipkart.com" + product.find('a', {'class': 'IRpwTa'})['href']
                brand_name = product.find('div', {'class': '_4rR01T'}).text.strip()
                smartphone_name = product.find('a', {'class': 'IRpwTa'}).text.strip()
                price = product.find('div', {'class': '_30jeq3'}).text.strip()
                details = product.find_all('li', {'class': 'rgWa7D'})

                color = details[0].text if len(details) > 0 else "-"
                ram = details[1].text if len(details) > 1 else "-"
                rom = details[2].text if len(details) > 2 else "-"
                primary_camera = details[3].text if len(details) > 3 else "-"
                secondary_camera = details[4].text if len(details) > 4 else "-"
                display_size = details[5].text if len(details) > 5 else "-"
                battery_capacity = details[6].text if len(details) > 6 else "-"

                smartphone_info = {
                    "Brand Name": brand_name,
                    "Smartphone Name": smartphone_name,
                    "Colour": color,
                    "RAM": ram,
                    "Storage(ROM)": rom,
                    "Primary Camera": primary_camera,
                    "Secondary Camera": secondary_camera,
                    "Display Size": display_size,
                    "Battery Capacity": battery_capacity,
                    "Price": price,
                    "Product URL": product_url
                }

                smartphone_data.append(smartphone_info)
            except Exception as e:
                print(f"Exception: {e}")

        df = pd.DataFrame(smartphone_data)
        df.to_csv(f"{product}_details.csv", index=False)

if __name__ == "__main__":
    user_input = input("Enter the smartphone you want to search for on Flipkart: ")
    scrape_flipkart_smartphones(user_input)


In [ ]:
#Q-5
import requests

def get_coordinates(city):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    api_key = "YOUR_API_KEY"  # Replace with your Google Maps API key
    params = {
        "address": city,
        "key": api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            latitude = location['lat']
            longitude = location['lng']
            print(f"Coordinates for {city}: Latitude {latitude}, Longitude {longitude}")
        else:
            print("Failed to fetch coordinates.")
    else:
        print("Failed to connect to the API.")

if __name__ == "__main__":
    city_input = input("Enter the city to get its coordinates: ")
    get_coordinates(city_input)


In [ ]:
#Q-6
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_digit_in():
    base_url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        laptops = soup.find_all('div', {'class': 'TopNumbeHeading active'})
        laptop_data = []

        for laptop in laptops:
            details = laptop.find_next('div', {'class': 'Right_TopBlock'})

            laptop_name = details.find('h3').text.strip()
            specs = details.find_all('p')
            specs_dict = {}
            for spec in specs:
                spec_text = spec.text.strip().split(":")
                if len(spec_text) == 2:
                    specs_dict[spec_text[0]] = spec_text[1]
                else:
                    specs_dict[spec_text[0]] = "-"

            laptop_data.append({
                "Laptop Name": laptop_name,
                "Processor": specs_dict.get("Processor", "-"),
                "OS": specs_dict.get("OS", "-"),
                "Display": specs_dict.get("Display", "-"),
                "Memory": specs_dict.get("Memory", "-"),
                "Weight": specs_dict.get("Weight", "-"),
                "Price": specs_dict.get("Price", "-")
            })

        df = pd.DataFrame(laptop_data)
        df.to_csv("gaming_laptops_digit_in.csv", index=False)

if __name__ == "__main__":
    scrape_digit_in()


In [ ]:
#Q-7
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_forbes_billionaires():
    base_url = "https://www.forbes.com/billionaires/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        billionaires = soup.find_all('div', {'class': 'personList'})

        billionaires_data = []
        for billionaire in billionaires:
            rank = billionaire.find('div', {'class': 'rank'}).text.strip()
            name = billionaire.find('div', {'class': 'personName'}).text.strip()
            net_worth = billionaire.find('div', {'class': 'netWorth'}).text.strip()
            age = billionaire.find('div', {'class': 'age'}).text.strip()
            citizenship = billionaire.find('div', {'class': 'countryOfCitizenship'}).text.strip()
            source = billionaire.find('div', {'class': 'source-column'}).text.strip()
            industry = billionaire.find('div', {'class': 'category'}).text.strip()

            billionaires_data.append({
                "Rank": rank,
                "Name": name,
                "Net Worth": net_worth,
                "Age": age,
                "Citizenship": citizenship,
                "Source": source,
                "Industry": industry
            })

        df = pd.DataFrame(billionaires_data)
        df.to_csv("forbes_billionaires.csv", index=False)

if __name__ == "__main__":
    scrape_forbes_billionaires()


In [ ]:
#Q-8
pip install google-api-python-client

from googleapiclient.discovery import build
import datetime

DEVELOPER_KEY = 'YOUR_API_KEY'
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def get_video_comments(video_id, max_results=500):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

    comments = []

    results = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=max_results
    ).execute()

    for item in results["items"]:
        comment = item["snippet"]["topLevelComment"]["snippet"]
        comment_text = comment["textDisplay"]
        like_count = comment.get("likeCount", 0)
        published_time = comment["publishedAt"]

        comments.append({
            "Comment": comment_text,
            "Upvotes": like_count,
            "Published Time": published_time
        })

    return comments

if __name__ == "__main__":
    # Provide a YouTube video ID
    video_id = 'YOUR_VIDEO_ID'
    video_comments = get_video_comments(video_id)

    for comment in video_comments:
        print(f"Comment: {comment['Comment']}")
        print(f"Upvotes: {comment['Upvotes']}")
        print(f"Published Time: {comment['Published Time']}")
        print("----------")


In [ ]:
#Q-9
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_hostels_in_london():
    base_url = "https://www.hostelworld.com/s/1/London/England"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        hostels = soup.find_all('div', {'class': 'hwta-property-list'})
        hostel_data = []

        for hostel in hostels:
            hostel_name = hostel.find('h2', {'class': 'title'}).text.strip()
            distance = hostel.find('span', {'class': 'description'}).text.strip()
            ratings = hostel.find('div', {'class': 'score orange'}).text.strip()
            total_reviews = hostel.find('div', {'class': 'reviews'}).text.strip()
            overall_reviews = hostel.find('div', {'class': 'keyword'}).text.strip()
            price_privates = hostel.find('div', {'class': 'price privates from'}).text.strip()
            price_dorms = hostel.find('div', {'class': 'price dorms from'}).text.strip()
            facilities = ", ".join([item.text.strip() for item in hostel.find_all('li', {'class': 'facility-badge'})])
            property_description = hostel.find('div', {'class': 'additional-info'}).text.strip()

            hostel_data.append({
                "Hostel Name": hostel_name,
                "Distance from City Centre": distance,
                "Ratings": ratings,
                "Total Reviews": total_reviews,
                "Overall Reviews": overall_reviews,
                "Privates From Price": price_privates,
                "Dorms From Price": price_dorms,
                "Facilities": facilities,
                "Property Description": property_description
            })

        df = pd.DataFrame(hostel_data)
        df.to_csv("hostels_in_london.csv", index=False)

if __name__ == "__main__":
    scrape_hostels_in_london()
